# Dartboard RAG: Retrieval-Augmented Generation with Balanced Relevance and Diversity

## Overview
The **Dartboard RAG** process addresses a common challenge in large knowledge bases: ensuring the retrieved information is both relevant and non-redundant. By explicitly optimizing a combined relevance-diversity scoring function, it prevents multiple top-k documents from offering the same information. This approach is drawn from the paper:

> [*Better RAG using Relevant Information Gain*](https://arxiv.org/abs/2407.12101)

The paper outlines three variations of the core idea—hybrid RAG (dense + sparse), a cross-encoder version, and a vanilla approach. The **vanilla approach** conveys the fundamental concept most directly, and this implementation extends it with optional weights to control the balance between relevance and diversity.

## Motivation

1. **Dense, Overlapping Knowledge Bases**  
   In large databases, documents may repeat similar content, causing redundancy in top-k retrieval.

2. **Improved Information Coverage**  
   Combining relevance and diversity yields a richer set of documents, mitigating the “echo chamber” effect of overly similar content.


## Key Components

1. **Relevance & Diversity Combination**  
   - Computes a score factoring in both how pertinent a document is to the query and how distinct it is from already chosen documents.

2. **Vanilla, Hybrid, or Cross-Encoder Variants**  
   - **Vanilla**: Uses basic similarity metrics to illustrate the core idea.  
   - **Hybrid**: Merges dense (e.g., cosine similarity) and sparse (e.g., BM25) vectors for improved coverage.  
   - **Cross-Encoder**: Incorporates cross-encoder scores for finer-grained document ranking, potentially with scaling or normalization.

3. **Weighted Balancing**  
   - Introduces \(\alpha\) (relevance weight) and \(\beta\) (diversity weight) to allow dynamic control of scoring.  
   - Helps in avoiding overly diverse but less relevant results.

4. **Production-Ready Code**  
   - Derived from the official implementation yet reorganized for clarity.  
   - Allows easier integration into existing RAG pipelines.

## Method Details

1. **Document Retrieval**  
   - Obtain an initial set of candidate documents based on similarity (e.g., cosine or BM25).  
   - Typically retrieves top-N candidates as a starting point.

2. **Scoring & Selection**  
   - Each document’s overall score combines **relevance** and **diversity**:  
   - Select the highest-scoring document, then penalize documents that are overly similar to it.  
   - Repeat until top-k documents are identified.

3. **Hybrid & Cross-Encoder Support**  
   - For **hybrid retrieval**: Merge similarity scores (dense and sparse) into the relevance term.  
   - For **cross-encoders**: Incorporate their ranking output into the relevance calculation, potentially adjusting with scaling factors.

4. **Balancing & Adjustment**  
   - Tune \(\alpha\) and \(\beta\) based on domain-specific needs.  
   - For example, if your domain values wide-ranging perspectives, increase \(\beta\); if precision is critical, emphasize \(\alpha\).



By integrating both **relevance** and **diversity** into retrieval, the Dartboard RAG approach ensures that top-k documents collectively offer richer, more comprehensive information—leading to higher-quality responses in Retrieval-Augmented Generation systems.



A practical solution is to **combine both relevance and diversity** into a single scoring function and directly optimize for it.
 
This is an implementation of the "dartboard" algorithm, as described in the following paper:

> [*Better RAG using Relevant Information Gain*](https://arxiv.org/abs/2407.12101)
(very elegant method, recommended reading)

The paper actually presents three variations on the same core idea; one with hybrid rag (dense and sparse), one with cross-encoder, and a vanilla version. The vanilla one conveys the idea, and is given here. If you have a hybrid RAG, you can just calculate cos-sim on both vectors and combine them for a similarity score. the shift from cross-encoder scores is straightforward too, but you might want some scaling of the distances.  

Additionally, I've introduced weights to control the balance between diversity and relevance.  
In real life, this weighting might help avoid retrieving overly diverse (and potentially less relevant) results.
The official paper also has a code implemention, and this code is based on it, but I think this one here is more readable, manageable and production ready.

### Import libraries and environment variables

In [2]:
import os
import sys
from dotenv import load_dotenv
import numpy as np
from scipy.special import logsumexp
from typing import Tuple, List
# Load environment variables from a .env file
load_dotenv()
# Set the OpenAI API key environment variable (comment out if not using OpenAI)
if not os.getenv('OPENAI_API_KEY'):
    print("Please enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = input("Please enter your OpenAI API key: ")
else:
    os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path since we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *


Please enter your OpenAI API key: 


### Read Docs

In [3]:
path = "../data/Understanding_Climate_Change.pdf"

### Encode document

In [4]:
# this part is same like simple_rag.ipynb, only simulating a dense dataset
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()
    documents=documents*5 # load every document 5 times to emulate a dense dataset

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings (Tested with OpenAI and Amazon Bedrock)
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.OPENAI)
    #embeddings = get_langchain_embedding_provider(EmbeddingProvider.AMAZON_BEDROCK)

    # Create vector store
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

### Create Vector store


In [5]:
chunks_vector_store = encode_pdf(path, chunk_size=1000, chunk_overlap=200)

### Some helper functions for using the vector store for retrieval.
this part is same like simple_rag.ipynb, only its using the actual FAISS index (not the wrapper)

In [6]:

def idx_to_text(idx:int):
    """
    Convert a Vector store index to the corresponding text.
    """
    docstore_id = chunks_vector_store.index_to_docstore_id[idx]
    document = chunks_vector_store.docstore.search(docstore_id)
    return document.page_content


def get_context(query:str,k:int=5) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Retrieve top k context items for a query using top k retrieval.
    """
    # regular top k retrieval
    q_vec=chunks_vector_store.embedding_function.embed_documents([query])
    _,indices=chunks_vector_store.index.search(np.array(q_vec),k=k)

    texts = [idx_to_text(i) for i in indices[0]]
    return texts


In [10]:

test_query = "What is the main cause of climate change?"


### Regular top k retrieval
- This demonstration shows that when database is dense (here we simulate density by loading each document 5 times), the results are not good, we don't get the most relevant results. Note that the top 3 results are all repetitions of the same document.

In [11]:
texts=get_context(test_query,k=3)
show_context(texts)

Context 1:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today.  
Coal


Context 2:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: C


### More utils for distances normalization

In [21]:
def lognorm(dist:np.ndarray, sigma:float):
    """
    Calculate the log-normal probability for a given distance and sigma.
    """
    if sigma < 1e-9: 
        return -np.inf * dist
    return -np.log(sigma) - 0.5 * np.log(2 * np.pi) - dist**2 / (2 * sigma**2)



### Definitions of parameters, and the actual function that optimizes both relevance and diversity 
This is the core function that chooses the top k documents based on relevance and diversity. It uses distances between each candidate document and the query and between candidate documents.

In [18]:

# Adjust these according to your needs, knowledge base density, etc. 
DIVERSITY_WEIGHT=1.0
RELEVANCE_WEIGHT=1.0
SIGMA=0.1


def greedy_dartsearch(q_dists:np.ndarray, dists_mat:np.ndarray, texts:List[str], k:int) -> Tuple[List[str], List[float]]:
    """
    Perform greedy dartboard search to select top k documents.
    """
    sigma=np.max([SIGMA,1e-5]) # avoid division by zero
    qprobs = lognorm(q_dists, sigma)
    ccprobmat = lognorm(dists_mat, sigma)
    out_scores=[]
    top_idx = np.argmax(qprobs) # start with the most relevant document
    chosen_inds = np.array([top_idx]) # initialize the array of selected documents
    maxes = ccprobmat[top_idx] # Vector of distances to the most relevant document
    while len(chosen_inds) < k:
        newmaxes = np.maximum(maxes, ccprobmat) # update the maximum distances, note the broadcasting (matrix and vector)

        logscores = newmaxes*DIVERSITY_WEIGHT + qprobs*RELEVANCE_WEIGHT # score all the items
        scores = logsumexp(logscores, axis=1) # normalize the scores
        scores[chosen_inds] = -np.inf # avoid selecting the same document twice
        best_idx = np.argmax(scores) # select the best item
        best_score=np.max(scores) # avoid division by zero
        maxes = newmaxes[best_idx] # update the maximum distances
        chosen_inds = np.append(chosen_inds, best_idx) # add the best item to the set
        out_scores.append(best_score) # add the best score to the list
    return [texts[i] for i in chosen_inds],out_scores



### Main function for using the dartboard retrieval. This serves instead of get_context (which is simple RAG) it:

1. Takes a text query, vectorzes it, gets the top k documents (and their vectors) via simple RAG. 
2. Uses these vectors to calculate the similarities to query and between candidate matches.
3. Runs the dartboard algorithm to refine the candidate matches to a final list of k documents.
4. Returns the final list of documents and their scores.

In [19]:

def get_context_with_dartboard(query:str,k:int=5) -> Tuple[List[str], List[float]]:
    """
    Retrieve top k context items for a query using the dartboard algorithm.
    This function only handles the vectors and indices, the rest is handled by the get_dartboard function and below.
    """
    q_vec=chunks_vector_store.embedding_function.embed_documents([query]) # embed the query
    _,indices=chunks_vector_store.index.search(np.array(q_vec),k=k*3) # fetch more than k to ensure we overcome density and use diversity

    vecs = np.array(chunks_vector_store.index.reconstruct_batch(indices[0])) # reconstruct the vectors of the retrieved documents
    texts = [idx_to_text(i) for i in indices[0]] # convert the indices to texts

    
    # calculate similarities and convert them to distances:
    dists_mat = 1-np.dot(vecs,vecs.T) # 1-cosine distance, you may think of better distance functions. This can also be applied to cross-encoder scores. 
    q_dists = 1-np.dot(q_vec,vecs.T) # calculate the distances to the query
    
    # run the dartboard algorithm
    texts, scores=greedy_dartsearch(q_dists,dists_mat,texts,k)
    return texts,scores


### dartboard retrieval - results on same query, k, and dataset
- As you can see now the top 3 results are not mere repetitions. 

In [22]:
texts,scores=get_context_with_dartboard(test_query,k=3)
show_context(texts)


Context 1:
driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today.  
Coal


Context 2:
Most of these climate changes are attributed to very small variations in Earth's orbit tha